In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
from ML import *
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print("GPU is","avaliable" if tf.config.experimental.list_physical_devices("GPU") else "not available")

GPU is not available


In [2]:
'''

We first import the necessary libraries and LOAD all the data for training and testing. Such data corresponds to the original samples from DeltaPhish (either benign or malicious) as well as the 100 phishing samples (not used in the training data) that we use as basis to create our adversarial examples.

We then develop a given leaner (i.e., a phishing detector) by using the train data and a Logistic Regression algorithm, and test it on the test data, computing the FPR and TPR shown in Table 3; we also compute the TPR on the subset of 100 samples used for our adversarial evaluations, which correspond to the results in Figure 4b (for the LR algorithm).

We then proceed to assess the robustness of the detector against ALL the 12 attacks considered in our paper, all of which use the 100 samples as basis. For WA and WA+, we already provide the preprocessed version. For PA, we use the samples created during the preprocessing phase. For MA, we create the samples at runtime.
'''

'\n\nWe first import the necessary libraries and LOAD all the data for training and testing. Such data corresponds to the original samples from DeltaPhish (either benign or malicious) as well as the 100 phishing samples (not used in the training data) that we use as basis to create our adversarial examples.\n\nWe then develop a given leaner (i.e., a phishing detector) by using the train data and a Logistic Regression algorithm, and test it on the test data, computing the FPR and TPR shown in Table 3; we also compute the TPR on the subset of 100 samples used for our adversarial evaluations, which correspond to the results in Figure 4b (for the LR algorithm).\n\nWe then proceed to assess the robustness of the detector against ALL the 12 attacks considered in our paper, all of which use the 100 samples as basis. For WA and WA+, we already provide the preprocessed version. For PA, we use the samples created during the preprocessing phase. For MA, we create the samples at runtime.\n'

In [3]:
# load data and split test and train data
json_file='../data_folder/deltaphish/preprocessed/normal/deltaphish_total_feature_modified.json'
data_delta=data_load(json_file)
# remove evasion dataset, then build new PWDs using remain data
phish_sub_test_x=joblib.load('../data_folder/deltaphish/preprocessed/phish_sub_test_x_100.pkl')
#print('phish_test_x index',phish_sub_test_x.index)#100
#print('phish_sub_test_x shape',phish_sub_test_x.shape)
data_delta_change=data_delta.copy()
#get the 100 phish samples and add them to the test dataset
phish_sub_test_later=data_delta_change.loc[phish_sub_test_x.index].copy()
data_delta_change=data_delta_change.drop(phish_sub_test_x.index)# del those 100 samples

train_random,test_random=train_test_split(data_delta_change,train_size=3200,test_size=700,shuffle=True,random_state=11)
train_x0=train_random.iloc[:,0:58]
train_y0=train_random.iloc[:,58]

#test_later=pd.concat(test_random,phish_sub_test_later)
frames =[test_random,phish_sub_test_later]
test_later=pd.concat(frames)
test_x0=test_later.iloc[:,0:58] 
test_y0=test_later.iloc[:,58] 

#use only html features
train_html_x0=train_random.iloc[:,27:49]
test_html_x0=test_later.iloc[:,27:49]

#only url features
train_url_x0=pd.concat([train_random.iloc[:,0:27],train_random.iloc[:,49:58]],axis=1)
test_url_x0=pd.concat([test_later.iloc[:,0:27],test_later.iloc[:,49:58]],axis=1)
train_y1=train_url_y0=train_html_y0=train_y0
test_y1=test_url_y0=test_html_y0=test_y0


benign_index=test_y0[test_y0==0].index.tolist()#
phish_index=test_y0[test_y0==1].index.tolist()#

#generate benign_test_x_y
benign_test_x=test_x0.loc[benign_index,:]
benign_test_y= test_y0.loc[benign_index]# 
benign_test_url_x=pd.concat([benign_test_x.iloc[:,0:27],benign_test_x.iloc[:,49:58]],axis=1)
benign_test_html_x=benign_test_x.iloc[:,27:49]

phish_sub_test_y=test_y0[phish_sub_test_x.index]#100 phishing samples
phish_sub_test_html_x=phish_sub_test_x.iloc[:,27:49]
phish_sub_test_url_x=pd.concat([phish_sub_test_x.iloc[:,0:27],phish_sub_test_x.iloc[:,49:58]],axis=1)

 

# baseline performance- refer to Table 3, bottom right cell

In [4]:
model_lr_0 = LogisticRegression(max_iter=1000,solver='liblinear',penalty='l1') 
model_lr_1 = LogisticRegression(max_iter=1000,solver='liblinear',penalty='l1') 
model_lr_2 = LogisticRegression(max_iter=1000,solver='liblinear',penalty='l1') 

model_lr_com,fpr_ave_com_lr,fpr_std_com_lr,recall_com_lr,re_std_com_lr,time1_com_lr,rfe_selector_lr_com =rfe_get_score(model_lr_0,41,train_x0,train_y0,test_x0,test_y0)
model_lr_url,fpr_ave_url_lr,fpr_std_url_lr,recall_url_lr,re_std_url_lr,time1_url_lr,rfe_selector_lr_url=rfe_get_score(model_lr_1,26,train_url_x0,train_url_y0,test_url_x0,test_url_y0)
model_lr_html,fpr_ave_html_lr,fpr_std_html_lr,recall_html_lr,re_std_html_lr,time1_html_lr,rfe_selector_lr_html=rfe_get_score(model_lr_2,15,train_html_x0,train_html_y0,test_html_x0,test_html_y0)
print('combine features, whole:')
lr_com_recall_whole=get_sub_base_rf_recall(model_lr_com,test_x0,test_y0,rfe_selector_lr_com)
print('url features, whole:')
lr_url_recall_whole=get_sub_base_rf_recall(model_lr_url,test_url_x0,test_url_y0,rfe_selector_lr_url)
print('html features, whole:')
lr_html_recall_whole=get_sub_base_rf_recall(model_lr_html,test_html_x0,test_html_y0,rfe_selector_lr_html)
 
print('lr_baseline_fpr-combine:' )
base_lf_fpr_com=get_base_rf_fpr(model_lr_com,benign_test_x,benign_test_y,rfe_selector_lr_com)
print('lr_baseline_fpr-url:')
base_lr_fpr_url=get_base_rf_fpr(model_lr_url,benign_test_url_x,benign_test_y,rfe_selector_lr_url)
print('lr_baseline_fpr-html:')
base_lr_fpr_html=get_base_rf_fpr(model_lr_html,benign_test_html_x,benign_test_y,rfe_selector_lr_html)

combine features, whole:
recall 0.84
url features, whole:
recall 0.26
html features, whole:
recall 0.77
lr_baseline_fpr-combine:
fpr 0.010
lr_baseline_fpr-url:
fpr 0.010
lr_baseline_fpr-html:
fpr 0.020


# no-atk,refer to Fig 4b, the green bars in the rightmost subplot

In [5]:
print('lr_no-atk_recall_combine:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x,phish_sub_test_y,rfe_selector_lr_com)
print('lr_no-atk_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x,phish_sub_test_y,rfe_selector_lr_url)
print('lr_no-atk_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x,phish_sub_test_y,rfe_selector_lr_html)
 

lr_no-atk_recall_combine:
recall 0.87
lr_no-atk_recall_url:
recall 0.32
lr_no-atk_recall_html:
recall 0.79


# wa_u attack- refer to Fig 4b, the orange bars in the rightmost subplot

In [6]:
ad_url_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_u/phish_sub_problem_space_url_shorten100.json")
ad_url_problem_test_x=ad_url_problem_data.iloc[:,0:58]
ad_url_problem_test_y=ad_url_problem_data.iloc[:,58]
#important to normalization 
ad_url_problem_test_url_x=pd.concat([ad_url_problem_data.iloc[:,0:27],ad_url_problem_data.iloc[:,49:58]],axis=1)
ad_url_problem_test_html_x=ad_url_problem_data.iloc[:,27:49]
#...............
print('lr_recall_combined_wa_u:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_url_problem_test_x,ad_url_problem_test_y,rfe_selector_lr_com)
print('lr_recall_url_wa_u:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_url_problem_test_url_x,ad_url_problem_test_y,rfe_selector_lr_url)
print('lr_recall_html_wa_u:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_url_problem_test_html_x,ad_url_problem_test_y,rfe_selector_lr_html)



lr_recall_combined_wa_u:
recall 0.58
lr_recall_url_wa_u:
recall 0.00
lr_recall_html_wa_u:
recall 0.79


# wa_r attack- refer to Fig 4b, the firebrick bars in the rightmost subplot

In [7]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_r/phish_sub_problem_space_html_add50internallinks.json")#wa_u,wa_c
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

print('lr_wa_r_recall_combined:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_problem_test_x,ad_problem_test_y,rfe_selector_lr_com)
print('lr_wa_r_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_problem_test_url_x,ad_problem_test_y,rfe_selector_lr_url)
print('lr_wa_r_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_problem_test_html_x,ad_problem_test_y,rfe_selector_lr_html)


lr_wa_r_recall_combined:
recall 0.78
lr_wa_r_recall_url:
recall 0.32
lr_wa_r_recall_html:
recall 0.60


# wa_c attack- refer to Fig 4b, the darkred bars in the rightmost subplot

In [8]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_c/phish_sub_problem_space_html_add50internallinks_combined_shorten_url.json")
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]
 
ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]
print('lr_wa_c_recall_combined:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_problem_test_x,ad_problem_test_y,rfe_selector_lr_com)
print('lr_wa_c_recall-url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_problem_test_url_x,ad_problem_test_y,rfe_selector_lr_url)
print('lr_wa_c_recall-html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_problem_test_html_x,ad_problem_test_y,rfe_selector_lr_html)



lr_wa_c_recall_combined:
recall 0.45
lr_wa_c_recall-url:
recall 0.00
lr_wa_c_recall-html:
recall 0.59


# wa+_u attack- refer to Fig 5b, top row,  green bars of the leftmost group of bars

In [9]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_u/phish_sub_html_change_full_knowledge_change_again_later_only_url_shorten.json")
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

print('lr_wa+_u_recall_combined:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_problem_test_x,ad_problem_test_y,rfe_selector_lr_com)
print('lr_wa+_u_recall-url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_problem_test_url_x,ad_problem_test_y,rfe_selector_lr_url)
print('lr_wa+_u_recall-html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_problem_test_html_x,ad_problem_test_y,rfe_selector_lr_html)



lr_wa+_u_recall_combined:
recall 0.82
lr_wa+_u_recall-url:
recall 0.00
lr_wa+_u_recall-html:
recall 0.79


# wa+_r attack- refer to Fig 5b, top row,  blue bars of the leftmost group of bars

In [10]:

ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_r/phish_sub_problem_space__full_onlyhtml_316.json")#wa+_u,wa+_c

ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]
 
print('lr_sub_baseline_recall_combined:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_problem_test_x,ad_problem_test_y,rfe_selector_lr_com)
print('lr_sub_baseline_recall-url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_problem_test_url_x,ad_problem_test_y,rfe_selector_lr_url)
print('lr_sub_baseline_recall-html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_problem_test_html_x,ad_problem_test_y,rfe_selector_lr_html)


lr_sub_baseline_recall_combined:
recall 0.72
lr_sub_baseline_recall-url:
recall 0.32
lr_sub_baseline_recall-html:
recall 0.62


# wa+_c attack- refer to Fig 5b, top row, red bars  of the leftmost group of bars

In [11]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_c/phish_sub_problem_space_mix_url_html_316.json")
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

print('lr_wa+_c_recall_combined:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_problem_test_x,ad_problem_test_y,rfe_selector_lr_com)
print('lr_wa+_c_recall-url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_problem_test_url_x,ad_problem_test_y,rfe_selector_lr_url)
print('lr_wa+_c_recall-html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_problem_test_html_x,ad_problem_test_y,rfe_selector_lr_html)



lr_wa+_c_recall_combined:
recall 0.75
lr_wa+_c_recall-url:
recall 0.00
lr_wa+_c_recall-html:
recall 0.66


# pa_u attack- refer to Fig 5b, top row, green bars  in middle group of bars

In [12]:

phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_u.pkl")
phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


print('lr_pa_u_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_pa_u_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_pa_u_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)



lr_pa_u_recall_com:
recall 0.96
lr_pa_u_recall_url:
recall 0.94
lr_pa_u_recall_html:
recall 0.81


# pa_r attack- refer to Fig 5b, top row, blue bars in middle group of bars

In [13]:
phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_r.pkl")
phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


print('lr_pa_r_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_pa_r_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_pa_r_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)


lr_pa_r_recall_com:
recall 0.24
lr_pa_r_recall_url:
recall 0.32
lr_pa_r_recall_html:
recall 0.04


# pa_c attack- refer to Fig 5b, top row, red bars in middle group of bars

In [14]:
phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_c.pkl")
phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


print('lr_pa_c_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_pa_c_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_pa_c_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)


lr_pa_c_recall_com:
recall 0.76
lr_pa_c_recall_url:
recall 0.94
lr_pa_c_recall_html:
recall 0.44


# ma_u attack- refer to Fig 5b, top row, green bars  of the rightmost group of bars

In [15]:
phish_sub_test_x_ch=phish_sub_test_x.copy()
phish_sub_test_x_ch['URL_length']=-1
phish_sub_test_x_ch['URL_IP']=-1
phish_sub_test_x_ch['URL_redirect']=-1
phish_sub_test_x_ch['URL_shortener']=1
phish_sub_test_x_ch['URL_subdomains']=-1
phish_sub_test_x_ch['URL_at']=-1 
phish_sub_test_x_ch['URL_dash']=-1  
phish_sub_test_x_ch['URL_numberofCommonTerms']=-1 
phish_sub_test_x_ch['URL_checkNumerical']=-1 
phish_sub_test_x_ch['URL_checkPathExtend']=-1
phish_sub_test_x_ch['URL_checkPunycode']=-1
phish_sub_test_x_ch['URL_checkSensitiveWord']=0
phish_sub_test_x_ch['URL_checkTLDinSub']=-1
phish_sub_test_x_ch['URL_checkTLDinPath']=-1
phish_sub_test_x_ch['URL_totalWordUrl']=0
 
phish_sub_test_x_ch['URL_longestWordUrl']=0
phish_sub_test_x_ch['URL_longestWordHost']=0
phish_sub_test_x_ch['URL_longestWordPath']=0
phish_sub_test_x_ch['URL_averageWordUrl']=1000
phish_sub_test_x_ch['URL_averageWordHost']=0
phish_sub_test_x_ch['URL_averageWordPath']=1000
phish_sub_test_x_ch['REP_checkGI']=-1

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

print('lr_ma_u_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_ma_u_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_ma_u_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)



lr_ma_u_recall_com:
recall 0.00
lr_ma_u_recall_url:
recall 0.00
lr_ma_u_recall_html:
recall 0.79


# ma_r attack- refer to Fig 5b, top row, blue bars  of the rightmost group of bars

In [16]:
phish_sub_test_x_ch=phish_sub_test_x.copy()

phish_sub_test_x_ch['HTML_Objects']=-1
phish_sub_test_x_ch['HTML_metaScripts']=-1
phish_sub_test_x_ch['HTML_FrequentDomain']=-1
phish_sub_test_x_ch['HTML_Commonpage']=1#0#1
phish_sub_test_x_ch['HTML_CommonPageRatioinFooter']=1#1
phish_sub_test_x_ch['HTML_popUp']=-1
phish_sub_test_x_ch['HTML_RightClick']=-1
phish_sub_test_x_ch['HTML_DomainwithCopyright']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_div']=-1
phish_sub_test_x_ch['HTML_nullLinksinWeb']=1
phish_sub_test_x_ch['HTML_nullLinksinFooter']=0#1
phish_sub_test_x_ch['HTML_BrokenLink']=1
phish_sub_test_x_ch['HTML_LoginForm']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_input']=-1
phish_sub_test_x_ch['HTML_TitleUrlBrand']=-1
phish_sub_test_x_ch['HTML_favicon']=-1
phish_sub_test_x_ch['HTML_statusBarMod']=-1
phish_sub_test_x_ch['HTML_css']=-1
phish_sub_test_x_ch['HTML_anchors']=-1

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


print('lr_ma_r_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_ma_r_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_ma_r_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)


lr_ma_r_recall_com:
recall 0.00
lr_ma_r_recall_url:
recall 0.32
lr_ma_r_recall_html:
recall 0.00


# ma_c attack- refer to Fig 5b, top row, red bars  of the rightmost group of bars

In [17]:
phish_sub_test_x_ch=phish_sub_test_x.copy()
phish_sub_test_x_ch['URL_length']=-1
phish_sub_test_x_ch['URL_IP']=-1#single no change
phish_sub_test_x_ch['URL_redirect']=-1#
phish_sub_test_x_ch['URL_shortener']=1 #single no change
phish_sub_test_x_ch['URL_subdomains']=-1
phish_sub_test_x_ch['URL_at']=-1 
phish_sub_test_x_ch['URL_dash']=-1  
phish_sub_test_x_ch['URL_numberofCommonTerms']=-1 
phish_sub_test_x_ch['URL_checkNumerical']=-1 #single no change
phish_sub_test_x_ch['URL_checkPathExtend']=-1#single no change
phish_sub_test_x_ch['URL_checkPunycode']=-1#single no change
phish_sub_test_x_ch['URL_checkSensitiveWord']=0
phish_sub_test_x_ch['URL_checkTLDinSub']=-1
phish_sub_test_x_ch['URL_checkTLDinPath']=-1
phish_sub_test_x_ch['URL_totalWordUrl']=0

phish_sub_test_x_ch['URL_longestWordUrl']=0
phish_sub_test_x_ch['URL_longestWordHost']=0 
phish_sub_test_x_ch['URL_longestWordPath']=0
phish_sub_test_x_ch['URL_averageWordUrl']=1000
phish_sub_test_x_ch['URL_averageWordHost']=0
phish_sub_test_x_ch['URL_averageWordPath']=1000
 
phish_sub_test_x_ch['HTML_Objects']=-1
phish_sub_test_x_ch['HTML_metaScripts']=-1
phish_sub_test_x_ch['HTML_FrequentDomain']=-1
phish_sub_test_x_ch['HTML_Commonpage']=1#0#1
phish_sub_test_x_ch['HTML_CommonPageRatioinFooter']=1#1
 
phish_sub_test_x_ch['HTML_popUp']=-1
phish_sub_test_x_ch['HTML_RightClick']=-1
phish_sub_test_x_ch['HTML_DomainwithCopyright']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_div']=-1
phish_sub_test_x_ch['HTML_nullLinksinWeb']=1#1#0
phish_sub_test_x_ch['HTML_nullLinksinFooter']=0#1
phish_sub_test_x_ch['HTML_BrokenLink']=1
phish_sub_test_x_ch['HTML_LoginForm']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_input']=-1
phish_sub_test_x_ch['HTML_TitleUrlBrand']=-1
phish_sub_test_x_ch['HTML_favicon']=-1
phish_sub_test_x_ch['HTML_statusBarMod']=-1
phish_sub_test_x_ch['HTML_css']=-1
phish_sub_test_x_ch['HTML_anchors']=-1
phish_sub_test_x_ch['REP_checkGI']=-1

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


print('lr_ma_c_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_ma_c_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_mc_c_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)



lr_ma_c_recall_com:
recall 0.00
lr_ma_c_recall_url:
recall 0.00
lr_mc_c_recall_html:
recall 0.00
